In [391]:
import torch
import torch.nn as nn
import torch.optim as optim
import math

In [392]:
class experiment(nn.Module):
    def __init__(self):
        super(experiment, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(3,4),
            nn.ReLU(),
        )
        self.fc2 = nn.Linear(4,3)
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

net = experiment()
net2 = experiment()
net3 = experiment()

for params in net.parameters():
    nn.init.normal_(params, mean=0, std=0.01)
    
net2.load_state_dict(net.state_dict())
net3.load_state_dict(net.state_dict())

print(net)

experiment(
  (fc1): Sequential(
    (0): Linear(in_features=3, out_features=4, bias=True)
    (1): ReLU()
  )
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)


In [393]:
print('net1 fc2.weight')
print(net.state_dict()['fc2.weight'])
print('net2 fc2.weight')
print(net2.state_dict()['fc2.weight'])
print('net3 fc2.weight')
print(net3.state_dict()['fc2.weight'])

net1 fc2.weight
tensor([[-0.0088,  0.0113, -0.0019, -0.0041],
        [ 0.0077, -0.0197, -0.0020,  0.0086],
        [ 0.0085, -0.0064, -0.0003,  0.0042]])
net2 fc2.weight
tensor([[-0.0088,  0.0113, -0.0019, -0.0041],
        [ 0.0077, -0.0197, -0.0020,  0.0086],
        [ 0.0085, -0.0064, -0.0003,  0.0042]])
net3 fc2.weight
tensor([[-0.0088,  0.0113, -0.0019, -0.0041],
        [ 0.0077, -0.0197, -0.0020,  0.0086],
        [ 0.0085, -0.0064, -0.0003,  0.0042]])


In [394]:
loss_func = nn.CrossEntropyLoss()
loss_func2 = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.1)
optimizer2 = optim.SGD(net2.parameters(), lr=0.1)
optimizer3 = optim.SGD(net3.parameters(), lr=0.1)

In [395]:
def train_with_rule(net, loss_func, optimizer, data, labels, PI):
    student_outputs = net(data) # outputs->[]    
    # print(student_outputs)
    student_loss = loss_func(student_outputs, labels.squeeze())
    teacher_loss = loss_func2(student_outputs, student_outputs*math.exp(0.5))
    loss = (1-PI) * student_loss + PI * teacher_loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(net.state_dict()['fc2.weight'])

In [396]:
def train_with_norule(net, loss_func, optimizer, data, labels):
    student_outputs = net(data) # outputs->[]    
    loss = loss_func(student_outputs, labels.squeeze()) # input(N,C) where C= number of classes Target(N)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(net.state_dict()['fc2.weight'])

In [397]:
x_input = torch.randn(50, 3)
y_target = torch.empty(50, dtype=torch.long).random_(3)
print('net1 fc2.weight')
train_with_rule(net, loss_func, optimizer, x_input, y_target, 0.8)
print('net2 fc2.weight')
train_with_norule(net2, loss_func, optimizer2, x_input, y_target)
print('net3 fc2.weight')
train_with_norule(net3, loss_func, optimizer3, x_input, y_target)

net1 fc2.weight
tensor([[-0.0088,  0.0114, -0.0019, -0.0041],
        [ 0.0077, -0.0197, -0.0020,  0.0086],
        [ 0.0085, -0.0065, -0.0004,  0.0042]])
net2 fc2.weight
tensor([[-0.0087,  0.0115, -0.0019, -0.0040],
        [ 0.0076, -0.0198, -0.0020,  0.0085],
        [ 0.0085, -0.0066, -0.0004,  0.0042]])
net3 fc2.weight
tensor([[-0.0087,  0.0115, -0.0019, -0.0040],
        [ 0.0076, -0.0198, -0.0020,  0.0085],
        [ 0.0085, -0.0066, -0.0004,  0.0042]])
